In [1]:
import tensorflow as tf
import os
import time

from model import *
from data_utils import *
from conf_utils import *

/Users/richardsun/anaconda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
class config:
    vocab_size = 2000
    embedding_size = 200
    filter_size = 64
    num_layers = 5
    block_size = 2
    filter_h = 5
    context_size = 20
    text_size = context_size
    batch_size = 32
    epochs = 5
    num_sampled = 1
    learning_rate = 1
    momentum = 0.99
    grad_clip = 0.1
    num_batches = 0
    ckpt_path = 'ckpt'
    summary_path = 'logs'
    data_dir = "data/texts/reviews/movie_reviews"

In [3]:
with open("data/texts/reviews/movie_reviews") as f:
    text = f.readlines()

In [4]:
conf = prepare_conf(config)

In [5]:
conf.text_size

22

In [6]:
x_batches, y_batches = prepare_data(conf)

In [7]:
x_batches[0][0].shape

(22,)

In [8]:
model = GatedCNN(conf)
saver = tf.train.Saver(tf.trainable_variables())
print("Started Model Training...")

Started Model Training...


In [9]:
batch_idx = 0
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(conf.summary_path, graph=sess.graph)

    if os.path.exists(conf.ckpt_file):
        saver.restore(sess, conf.ckpt_file)
        print("Model Restored")

    for i in np.arange(conf.epochs):
        start = time.time()
        for j in np.arange(conf.num_batches):
            inputs, labels, batch_idx = get_batch(x_batches, y_batches, batch_idx)
            _, l = sess.run([model.optimizer, model.loss], feed_dict={model.X:inputs, model.y:labels})
        end = time.time()
        print("Epoch: %.2f, Time: %.2f,  Loss: %.2f"%(i, end-start, l))

        if i % 10 == 0:
            perp = sess.run(model.perplexity, feed_dict={model.X:inputs, model.y:labels})
            print("Perplexity: %.2f"%perp)
            saver.save(sess, conf.ckpt_file)

        summaries = sess.run(model.merged_summary_op, feed_dict={model.X:inputs, model.y:labels})
        summary_writer.add_summary(summaries, i)

KeyboardInterrupt: 

In [9]:
from nltk.corpus import movie_reviews
reviews = open('movie_reviews', 'w+')
for file in movie_reviews.fileids():
    words = movie_reviews.words(file)
    if len(words) > config.context_size-2:
        words = words[:(config.context_size-2)]
        text = ' '.join(words)
        reviews.write(text+'\n')
reviews.close()